In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [122]:
df=pd.read_excel("Q1.xlsx")
df=df.sample(frac=1,random_state=42).reset_index(drop=True)
print(df)

X=df.drop("Y",axis=1).values.astype(float)
ones=np.ones((X.shape[0],1))
X=np.column_stack((X,ones))
Y=df["Y"].values.astype(float)
Y=np.expand_dims(Y,axis=1)
print(X)

       X1     X2     X3      X4   X5  X6    X7  X8      Y
0    0.62  808.5  367.5  220.50  3.5   2  0.40   3  16.47
1    0.66  759.5  318.5  220.50  3.5   2  0.25   1  13.17
2    0.98  514.5  294.0  110.25  7.0   2  0.40   3  32.82
3    0.79  637.0  343.0  147.00  7.0   4  0.40   4  41.32
4    0.64  784.0  343.0  220.50  3.5   3  0.25   4  16.69
..    ...    ...    ...     ...  ...  ..   ...  ..    ...
763  0.76  661.5  416.5  122.50  7.0   5  0.10   1  32.21
764  0.86  588.0  294.0  147.00  7.0   4  0.10   2  26.33
765  0.71  710.5  269.5  220.50  3.5   4  0.10   5  10.67
766  0.98  514.5  294.0  110.25  7.0   5  0.25   4  28.62
767  0.90  563.5  318.5  122.50  7.0   4  0.10   2  28.83

[768 rows x 9 columns]
[[6.200e-01 8.085e+02 3.675e+02 ... 4.000e-01 3.000e+00 1.000e+00]
 [6.600e-01 7.595e+02 3.185e+02 ... 2.500e-01 1.000e+00 1.000e+00]
 [9.800e-01 5.145e+02 2.940e+02 ... 4.000e-01 3.000e+00 1.000e+00]
 ...
 [7.100e-01 7.105e+02 2.695e+02 ... 1.000e-01 5.000e+00 1.000e+00]
 [9.800

In [123]:
def train_test_split(X,Y,split):
    # split belongs [0,1]
    split_num=int(Y.shape[0]*split)
    Y_1=Y[:split_num,:]
    Y_2=Y[split_num:,:]
    X_1=X[:split_num,:]
    X_2=X[split_num:,:]
    return X_1,X_2,Y_1,Y_2
# seperating traing set for 70%
x_train,xtem,y_train,ytem=train_test_split(X,Y,split=0.7)
print(x_train.shape,y_train.shape)
# seperating cross validation(15%) and testing set(15%) split=0.5
x_val,x_test,y_val,y_test=train_test_split(xtem,ytem,split=0.5)
print(x_test.shape,y_test.shape)

(537, 9) (537, 1)
(116, 9) (116, 1)


In [124]:
theta=np.zeros((X.shape[1],1)) # defined theta for parameters

def h(xi):
    xi=np.expand_dims(xi,axis=0)
    # xi will be an matrix so x_i.shape == (1,9)
    r=xi@theta
    return r[0][0]

def error(X,theta,Y):
    r=((X@theta-Y).T @ (X@theta-Y))/X.shape[0]
    return r[0][0]

print(h(x_train[0]),error(X,theta,Y))

0.0 599.2906108802084


Q1 part 1

In [127]:
XtX = x_train.T @ x_train
print("cond(X^T X) =", np.linalg.cond(XtX))
# conditioning value is much high so we will use np.linalg.pinv(X)=(X.T@X)@X.T it will give an pseudo value

cond(X^T X) = 2.098873410281493e+19


In [ ]:

theta_1 = np.linalg.pinv(x_train) @ y_train
print(theta_1,theta_1.shape)
err=error(x_train,theta_1,y_train)
print(err)
print(error(x_val,theta_1,y_val))
print(error(x_test,theta_1,y_test))


[[-6.93076023e+01]
 [-6.67350686e-02]
 [ 3.47384540e-02]
 [-5.07367618e-02]
 [ 4.23852278e+00]
 [-7.57597601e-02]
 [ 2.00831410e+01]
 [ 1.60475703e-01]
 [ 9.09310768e+01]] (9, 1)
8.810565450206356
7.9428421412048555
7.920323307311352


Q1 part 2

In [ ]:
class linear_regression_model:
    def __init__(self):
        self.weights=None
        self.alpha=None
        self.error_T=[]
        self.error_val=[]
        self.epochs=None
    def fit(xt,yt,xv,yv,alpha,epochs,self):
        self.weights=np.zeros((xt.shape[1],1)) # weight initialization
        self.alpha=alpha
        self.epochs=epochs
        for i in range(self.epochs):
            self.weights=self.weights-self.alpha*(xt.T@(xt@self.weights-yt))
            self.error_T.append(error(xt,self.weights,yt))
            self.error_val.append(error(xv,self.weights,yv))
